# <center> Исследование данных Samsung. Классификация физической активности пользователей

В этой работе мы продолжим наше исследование данных, взятых с датчиков акселерометров и гироскопов смартфонов Samsung Galaxy S3. Телефоны носили в кармане добровольцы в возрасте от 19 до 49 лет. Смартфоны постоянно фиксировали значения ускорения и скорости по трём измерениям, а поведение людей записывали на видео, чтобы вручную отметить, какую физическую активность осуществлял человек в тот или иной момент.

Данные содержат следующие признаки:
* различные показатели с акселерометра и гироскопа;
* метка активности (физическая активность человека в конкретный момент).

Попробуем на основе данных с гироскопа и акселерометра разделить активности людей на некоторые схожие по своим характеристикам группы с помощью алогиритмов классификации.

In [1]:
# Импорт библиотек
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import decomposition
from sklearn import model_selection
from sklearn import linear_model
from sklearn import multioutput
from sklearn import svm
from sklearn import neural_network
from sklearn import tree
from sklearn import ensemble
import xgboost as xgb
import catboost as cb
from sklearn import metrics
# Для оптимизации моделей
import optuna
# Для графики и отображения
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn-v0_8')
# Подавление варнов
import warnings
warnings.filterwarnings('ignore')

## Загрузка и обработка данных

In [2]:
X_train = np.loadtxt("data/train.txt")
y_train = np.loadtxt("data/train_labels.txt")
 
X_test = np.loadtxt("data/test.txt")
y_test = np.loadtxt("data/test_labels.txt")

print(f'Число наблюдений в трейне: {X_train.shape[0]}')
print(f'Число наблюдений в тесте:  {X_test.shape[0]}')
print(f'Число фич: {X_train.shape[1]}')

Число наблюдений в трейне: 7352
Число наблюдений в тесте:  2947
Число фич: 561


Найдем также число различных активностей — сколько наблюдений должны быть отнесены каждому классу в трейне и в тесте.

In [3]:
print(
    'Список лейблов целевой переменной: ',
    list(map(int, list(np.unique(y_train, return_counts=True)[0])))
)

print(
    'Список размеров истинных кластеров в трейне:',
    list(np.unique(y_train, return_counts=True)[1])
)

print(
    'Список размеров истинных кластеров в тесте:',
    list(np.unique(y_test, return_counts=True)[1])
)

Список лейблов целевой переменной:  [1, 2, 3, 4, 5, 6]
Список размеров истинных кластеров в трейне: [1226, 1073, 986, 1286, 1374, 1407]
Список размеров истинных кластеров в тесте: [496, 471, 420, 491, 532, 537]


Как видим, есть ряд активностей, обозначенных цифрами. Эти метки означают следующее:

- 1 — ходьба;
- 2 — подъём;
- 3 — спуск;
- 4 — сидение;
- 5 — стояние;
- 6 — лежание.

Далее необходимо отмасштабировать признаки. Будем использовать для этого алгоритм StandardScaler.

In [4]:
scaler = preprocessing.StandardScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.fit_transform(X_test)

Исследуем корреляцию, рассчитаем ранг и определитель матрицы. 

In [5]:
# Матрица корреляций
X_corr = np.corrcoef(X_train_norm.T)
# Параметры матрицы
print('Число столбцов:', X_corr.shape[1])
print('Ранг матрицы:  ', np.linalg.matrix_rank(X_corr))
print('Определитель:  ', np.linalg.det(X_corr))

Число столбцов: 561
Ранг матрицы:   470
Определитель:   0.0


Понизим размерность данных с помощью метода главных компонент.

In [6]:
# Определяем метод таким образом, чтобы отобранные им
# компоненты объясняли не менее 99.99 % разброса данных
pca = decomposition.PCA(n_components=0.9999)
# Преобразуем данные
X_train_pca = pca.fit_transform(X_train_norm)
X_test_pca = pca.transform(X_test_norm)
print(f'Число компонент: {len(X_train_pca[0])}')
# Матрица корреляций
corr_X_pca = np.corrcoef(X_train_pca.T)
# Параметры матрицы
print('Число столбцов:', corr_X_pca.shape[1])
print('Ранг матрицы:  ', np.linalg.matrix_rank(corr_X_pca))
print('Определитель:  ', np.linalg.det(corr_X_pca))

Число компонент: 372
Число столбцов: 372
Ранг матрицы:   372
Определитель:   0.9999999999999873


Удалось несколько уменьшить число признаков.

Объединим выборки для кроссвалидации.

In [86]:
X = np.concatenate((X_train_pca, X_test_pca))
y = np.concatenate((y_train, y_test))

## Моделирование

In [95]:
# Напишем одну полезную функцию
def metrics_extraction(preds_train, preds_test, train_y, test_y):
    """
    Функция для получения массива значений метрик для
    их дальнейшей печати и предачи в сводную таблицу.
    Args:
        preds_train (np.array): вектор предсказанных значений train-выборки;
        preds_test (np.array): вектор предсказанных значений test-выборки;
        train_y (np.array): вектор истинных значений train-выборки;
        test_y (np.array): вектор истинных значений test-выборки.
    Returns:
        tuple: 
            list: список строк для вывода метрик треннировочной выборки,
            list: список строк для вывода метрик тествовой выборки,
            array: вектор метрик треннировочной выборки,
            array: вектор метрик тествовой выборки.
    """
    
    precision_train = metrics.precision_score(train_y, preds_train, average='weighted')
    precision_test = metrics.precision_score(test_y, preds_test, average='weighted')
    recall_train = metrics.recall_score(train_y, preds_train, average='weighted')
    recall_test = metrics.recall_score(test_y, preds_test, average='weighted')
    f1_train = metrics.f1_score(train_y, preds_train, average='weighted')
    f1_test = metrics.f1_score(test_y, preds_test, average='weighted')
    accuracy_train = metrics.accuracy_score(train_y, preds_train)
    accuracy_test = metrics.accuracy_score(test_y, preds_test)
    
    train_print_lst = [
        'Train aggregate metrics:',
        f'Precision: {round(precision_train, 4)}',
        f'Recall:    {round(recall_train, 4)}',
        f'F1 score:  {round(f1_train, 4)}',
        f'Accuracy:  {round(accuracy_train, 4)}'
    ]
    
    test_print_lst = [
        'Test aggregate metrics:',
        f'Precision: {round(precision_test, 4)}',
        f'Recall:    {round(recall_test, 4)}',
        f'F1 score:  {round(f1_test, 4)}',
        f'Accuracy:  {round(accuracy_test, 4)}'
    ]
    
    train_array = np.array([
        precision_train, recall_train, f1_train, accuracy_train
    ])
    
    test_array = np.array([
        precision_test, recall_test, f1_test, accuracy_test
    ])
    
    return train_print_lst, test_print_lst, train_array, test_array


def cv_scores(model, X, y, cv=10):
    """
    Функция кросс-валидации модели.
    Args:
        model: объект, который будет обучаться на данных;
        X (np.array): матрица данных;
        y (np.array): вектор целевой переменной;
        cv (int, cross-validation generator):
            число фолдов кросс-валидации, либо cross-validation estimator.
    Returns:
        None
        prints:
           Train scores: precision, recall, f1_weighted.
           Test scores: precision, recall, f1_weighted.
    """
    scores = model_selection.cross_validate(
        model, X, y, cv=cv,
        scoring=('precision_weighted', 'recall_weighted', 'f1_weighted', 'accuracy'),
        return_train_score=True
    )
    print('Train scores:')
    print('Precision:  ', round(np.mean(scores['train_precision_weighted']), 4))
    print('Recall:     ', round(np.mean(scores['train_recall_weighted']), 4))
    print('F1 weighted:', round(np.mean(scores['train_f1_weighted']), 4))
    print('Accuracy:   ', round(np.mean(scores['train_accuracy']), 4))
    print('Test scores:')
    print('Precision:  ', round(np.mean(scores['test_precision_weighted']), 4))
    print('Recall:     ', round(np.mean(scores['test_recall_weighted']), 4))
    print('F1 weighted:', round(np.mean(scores['test_f1_weighted']), 4))
    print('Accuracy:   ', round(np.mean(scores['test_accuracy']), 4))

### Baseline model. CatBoostClassifier

In [52]:
%%time
# Инициализация объекта
cbc = cb.CatBoostClassifier(
    verbose=False,
    random_state=42
)
# Обучение
cbc.fit(X_train_pca, y_train)
# Предсказания
preds_train_cbc = cbc.predict(X_train_pca)
preds_test_cbc = cbc.predict(X_test_pca)
# Получение метрик
train_print_cbc, test_print_cbc, train_array_cbc, test_array_cbc = metrics_extraction(
    preds_train_cbc, preds_test_cbc, y_train, y_test
)
# Вывод метрик
print(*train_print_cbc, '-'*25, sep='\n')
print(*test_print_cbc, '-'*25, sep='\n')
print('Full test metrics report:')
print(metrics.classification_report(y_test, preds_test_cbc, digits=4))

Train aggregate metrics:
Precision: 1.0
Recall:    1.0
F1 score:  1.0
Accuracy:  1.0
-------------------------
Test aggregate metrics:
Precision: 0.9464
Recall:    0.9464
F1 score:  0.9462
Accuracy:  0.9464
-------------------------
Full test metrics report:
              precision    recall  f1-score   support

         1.0     0.9556    0.9536    0.9546       496
         2.0     0.9273    0.9745    0.9503       471
         3.0     0.9296    0.8810    0.9046       420
         4.0     0.9318    0.9185    0.9251       491
         5.0     0.9330    0.9417    0.9373       532
         6.0     0.9944    0.9963    0.9953       537

    accuracy                         0.9464      2947
   macro avg     0.9453    0.9443    0.9446      2947
weighted avg     0.9464    0.9464    0.9462      2947

CPU times: total: 17min 50s
Wall time: 1min 42s


Наличествует переобучение, но результаты уже весьма хорошие. Посмотрим, сможем ли мы их улучшить.

### LogisticRegression

In [53]:
%%time
# Напишем функцию для оптимизации
def optuna_LogR(trial):
    """Функция, обучающая модель LogisticRegression по переданным гиперпараметрам
    Args:
        trial : класс, от которого вызываются гиперпараметры
    Returns:
        score(float): метрика F1
    """
    # Задаем пространство поиска гиперпараметров
    params = {
        'penalty': trial.suggest_categorical('penalty', ['l2', None]),
        'C': trial.suggest_float('C', 0.1, 1),
        'solver': trial.suggest_categorical('solver', ['lbfgs', 'newton-cg', 'newton-cholesky'])
    }
    
    # Создаем модель
    model = linear_model.LogisticRegression(
        **params,
        max_iter=3000,
        random_state=42,
        n_jobs=-1
    )
    # Рассчитываем метрику на кросс-валидации
    score = model_selection.cross_val_score(
        model, X_train_pca, y_train, cv=10, scoring="f1_weighted", n_jobs=-1
    ).mean()
    return score

# Создаем объект исследования для первого набора гиперпараметров
# Укажем, что нам необходимо максимизировать метрику direction="maximize"
study_LogR = optuna.create_study(study_name="LogR_opt", direction="maximize",)
# Подавляем логирование
optuna.logging.set_verbosity(optuna.logging.WARNING)
# Ищем лучшую комбинацию гиперпараметров n_trials раз
study_LogR.optimize(optuna_LogR, n_trials=20)

# Передаем модели коллекцию оптимальных гиперпараметров
LogR_opt = linear_model.LogisticRegression(
    **study_LogR.best_params,
    max_iter=3000,
    random_state=42,
    n_jobs=-1
)
# Обучение
LogR_opt.fit(X_train_pca, y_train)
# Предсказания
preds_train_LogR_opt = LogR_opt.predict(X_train_pca)
preds_test_LogR_opt = LogR_opt.predict(X_test_pca)
# Получение метрик
train_print_LogR, test_print_LogR, train_array_LogR, test_array_LogR = metrics_extraction(
    preds_train_LogR_opt, preds_test_LogR_opt, y_train, y_test
)
# Вывод метрик
print(f'Лучшие гиперпараметры: {study_LogR.best_params}')
print("f1_score на обучающем наборе: {:.4f}\n".format(study_LogR.best_value))
print(*train_print_LogR, '-'*25, sep='\n')
print(*test_print_LogR, '-'*25, sep='\n')
print('Full test metrics report:')
print(metrics.classification_report(y_test, preds_test_LogR_opt, digits=4))

Лучшие гиперпараметры: {'penalty': 'l2', 'C': 0.7630868925337111, 'solver': 'newton-cholesky'}
f1_score на обучающем наборе: 0.9463

Train aggregate metrics:
Precision: 0.9942
Recall:    0.9942
F1 score:  0.9942
Accuracy:  0.9942
-------------------------
Test aggregate metrics:
Precision: 0.9658
Recall:    0.9654
F1 score:  0.9653
Accuracy:  0.9654
-------------------------
Full test metrics report:
              precision    recall  f1-score   support

         1.0     0.9779    0.9798    0.9789       496
         2.0     0.9684    0.9745    0.9714       471
         3.0     0.9833    0.9786    0.9809       420
         4.0     0.9585    0.8941    0.9252       491
         5.0     0.9144    0.9643    0.9387       532
         6.0     0.9963    1.0000    0.9981       537

    accuracy                         0.9654      2947
   macro avg     0.9665    0.9652    0.9655      2947
weighted avg     0.9658    0.9654    0.9653      2947

CPU times: total: 422 ms
Wall time: 2min 46s


Попробуем запустить логистическую регрессию без оптимизации.

In [72]:
# Передаем модели коллекцию оптимальных гиперпараметров
logr = linear_model.LogisticRegression(
    solver='newton-cholesky',
    random_state=42
)
# Обучение
logr.fit(X_train_pca, y_train)
# Предсказания
preds_train_logr = logr.predict(X_train_pca)
preds_test_logr = logr.predict(X_test_pca)
# Получение метрик
train_print_logr, test_print_logr, train_array_logr, test_array_logr = metrics_extraction(
    preds_train_logr, preds_test_logr, y_train, y_test
)
# Вывод метрик
print(*train_print_logr, '-'*25, sep='\n')
print(*test_print_logr, '-'*25, sep='\n')
print('Full test metrics report:')
print(metrics.classification_report(y_test, preds_test_logr, digits=4))

Train aggregate metrics:
Precision: 0.9951
Recall:    0.9951
F1 score:  0.9951
Accuracy:  0.9951
-------------------------
Test aggregate metrics:
Precision: 0.9659
Recall:    0.9654
F1 score:  0.9653
Accuracy:  0.9654
-------------------------
Full test metrics report:
              precision    recall  f1-score   support

         1.0     0.9779    0.9798    0.9789       496
         2.0     0.9684    0.9745    0.9714       471
         3.0     0.9833    0.9786    0.9809       420
         4.0     0.9605    0.8921    0.9250       491
         5.0     0.9130    0.9662    0.9388       532
         6.0     0.9963    1.0000    0.9981       537

    accuracy                         0.9654      2947
   macro avg     0.9665    0.9652    0.9655      2947
weighted avg     0.9659    0.9654    0.9653      2947



In [71]:
# создаём таблицу сопряжённости
ct = pd.crosstab(y_test, preds_test_LogR_opt)
# определяем название активностей
ct.index = ['ходьба', 'подъём', 
            'спуск', 'сидение',
            'стояние', 'лежание'
]
ct.columns = list(ct.index)
ct

,ходьба,подъём,спуск,сидение,стояние,лежание
ходьба,486,5,5,0,0,0
подъём,10,459,2,0,0,0
спуск,1,8,411,0,0,0
сидение,0,2,0,440,47,2
стояние,0,0,0,19,513,0
лежание,0,0,0,0,0,537


Пожалуй, можно уже заключить, что наша модель на основе логистической регрессии неплохо справилась с задачей и даже преодолела базовую (удивительный случай, т.к. дефолтный классификатор Catboost часто показывает лучший результат). Для оптимизации ошибки выбран метод Ньютона-Холецкого, мы также наблюдаем переобучение почти на три процента по всем агрегированным метрикам. Автоматический поиск оптимальной комбинации гиперпараметров не сильно улучшил результат.

Также стоит отметить распределение ошибок на таблице сопряженности: как и в случае кластеризации, мы видим почти четкое разделение на активные и пассивные действия — за исключением аномалии в виде отнесения двух "сидений" к классу "подъемов".

### PassiveAggressiveClassifier

In [54]:
pac = linear_model.PassiveAggressiveClassifier(
    random_state=42
)
# Обучение
pac.fit(X_train_pca, y_train)
# Предсказания
preds_train_pac = pac.predict(X_train_pca)
preds_test_pac = pac.predict(X_test_pca)
# Получение метрик
train_print_pac, test_print_pac, train_array_pac, test_array_pac = metrics_extraction(
    preds_train_pac, preds_test_pac, y_train, y_test
)
# Вывод метрик
print(*train_print_pac, '-'*25, sep='\n')
print(*test_print_pac, '-'*25, sep='\n')
print('Full test metrics report:')
print(metrics.classification_report(y_test, preds_test_pac, digits=4))

Train aggregate metrics:
Precision: 0.9917
Recall:    0.9917
F1 score:  0.9917
Accuracy:  0.9917
-------------------------
Test aggregate metrics:
Precision: 0.9578
Recall:    0.9559
F1 score:  0.956
Accuracy:  0.9559
-------------------------
Full test metrics report:
              precision    recall  f1-score   support

         1.0     0.9698    0.9698    0.9698       496
         2.0     0.9701    0.9660    0.9681       471
         3.0     0.9858    0.9929    0.9893       420
         4.0     0.9596    0.8697    0.9124       491
         5.0     0.8735    0.9737    0.9209       532
         6.0     0.9962    0.9665    0.9811       537

    accuracy                         0.9559      2947
   macro avg     0.9592    0.9564    0.9569      2947
weighted avg     0.9578    0.9559    0.9560      2947



Линейные алгоритмы, похоже, в целом неплохо справляются с задачей — по сравнению с дефолтным Catboost.

### Perceptron

In [13]:
pcn = linear_model.Perceptron(
    random_state=42
)
# Обучение
pcn.fit(X_train_pca, y_train)
# Предсказания
preds_train_pcn = pcn.predict(X_train_pca)
preds_test_pcn = pcn.predict(X_test_pca)
# Получение метрик
train_print_pcn, test_print_pcn, train_array_pcn, test_array_pcn = metrics_extraction(
    preds_train_pcn, preds_test_pcn, y_train, y_test
)
# Вывод метрик
print(*train_print_pcn, '-'*25, sep='\n')
print(*test_print_pcn, '-'*25, sep='\n')
print('Full test metrics report:')
print(metrics.classification_report(y_test, preds_test_pcn, digits=4))

Train aggregate metrics:
Precision: 0.9901
Recall:    0.9898
F1 score:  0.9898
Accuracy:  0.9898
-------------------------
Test aggregate metrics:
Precision: 0.9554
Recall:    0.9522
F1 score:  0.9522
Accuracy:  0.9522
-------------------------
Full test metrics report:
              precision    recall  f1-score   support

         1.0     0.9917    0.9657    0.9785       496
         2.0     0.9749    0.9894    0.9821       471
         3.0     0.9834    0.9881    0.9857       420
         4.0     0.9532    0.8289    0.8867       491
         5.0     0.8450    0.9737    0.9048       532
         6.0     0.9943    0.9702    0.9821       537

    accuracy                         0.9522      2947
   macro avg     0.9571    0.9527    0.9533      2947
weighted avg     0.9554    0.9522    0.9522      2947



### RidgeClassifier

In [32]:
ridge = linear_model.RidgeClassifier(
    solver='lsqr',
    random_state=42
)
# Обучение
ridge.fit(X_train_pca, y_train)
# Предсказания
preds_train_ridge = ridge.predict(X_train_pca)
preds_test_ridge = ridge.predict(X_test_pca)
# Получение метрик
train_print_ridge, test_print_ridge, train_array_ridge, test_array_ridge = metrics_extraction(
    preds_train_ridge, preds_test_ridge, y_train, y_test
)
# Вывод метрик
print(*train_print_ridge, '-'*25, sep='\n')
print(*test_print_ridge, '-'*25, sep='\n')
print('Full test metrics report:')
print(metrics.classification_report(y_test, preds_test_ridge, digits=4))

Train aggregate metrics:
Precision: 0.9826
Recall:    0.9826
F1 score:  0.9826
Accuracy:  0.9826
-------------------------
Test aggregate metrics:
Precision: 0.9628
Recall:    0.961
F1 score:  0.9612
Accuracy:  0.961
-------------------------
Full test metrics report:
              precision    recall  f1-score   support

         1.0     0.9778    0.9778    0.9778       496
         2.0     0.9624    0.9788    0.9705       471
         3.0     0.9976    0.9810    0.9892       420
         4.0     0.9650    0.8982    0.9304       491
         5.0     0.8821    0.9699    0.9239       532
         6.0     1.0000    0.9628    0.9810       537

    accuracy                         0.9610      2947
   macro avg     0.9641    0.9614    0.9621      2947
weighted avg     0.9628    0.9610    0.9612      2947



### SGDClassifier

Попробуем подобрать оптимальную loss-функцию к стохастическому градиентному спуску.

In [60]:
# Инициализация объекта
sgdc = linear_model.SGDClassifier(
    loss='perceptron',
    random_state=42
)
# Обучение
sgdc.fit(X_train_pca, y_train)
# Предсказания
sgdc_preds_train = sgdc.predict(X_train_pca)
sgdc_preds_test = sgdc.predict(X_test_pca)

# Получение метрик
train_print_sgdc, test_print_sgdc, train_array_sgdc, test_array_sgdc = metrics_extraction(
    sgdc_preds_train, sgdc_preds_test, y_train, y_test
)

# Вывод метрик
print(*train_print_sgdc, '-'*25, sep='\n')
print(*test_print_sgdc, '-'*25, sep='\n')
print('Full test metrics report:')
print(metrics.classification_report(y_test, sgdc_preds_test, digits=4))

Train aggregate metrics:
Precision: 0.9879
Recall:    0.9879
F1 score:  0.9879
Accuracy:  0.9879
-------------------------
Test aggregate metrics:
Precision: 0.9527
Recall:    0.9511
F1 score:  0.9513
Accuracy:  0.9511
-------------------------
Full test metrics report:
              precision    recall  f1-score   support

         1.0     0.9716    0.9657    0.9687       496
         2.0     0.9498    0.9639    0.9568       471
         3.0     0.9618    0.9595    0.9607       420
         4.0     0.9561    0.8880    0.9208       491
         5.0     0.8797    0.9624    0.9192       532
         6.0     1.0000    0.9665    0.9830       537

    accuracy                         0.9511      2947
   macro avg     0.9532    0.9510    0.9515      2947
weighted avg     0.9527    0.9511    0.9513      2947



Мы не стали включать в работу оптимизированный с помощью optuna алгоритм, т.к. нам не удалось подобрать с ним такие параметры, что показали бы лучший результат чем тот, что мы имеем в варианте с указанием лишь loss-функции: линейные потери, используемые алгоритмом перцептрона, в нашем кейсе минимизируются алгоритмом SGD эффективнее прочих. Базовая модель все же преодолена.

### MLPClassifier

In [55]:
# Инициализация объекта
mlpc = neural_network.MLPClassifier(
    hidden_layer_sizes=(166, 36),
    activation='tanh',
    random_state=42
)
# Обучение
mlpc.fit(X_train_pca, y_train)
# Предсказания
mlpc_preds_train = mlpc.predict(X_train_pca)
mlpc_preds_test = mlpc.predict(X_test_pca)

# Получение метрик
train_print_mlpc, test_print_mlpc, train_array_mlpc, test_array_mlpc = metrics_extraction(
    mlpc_preds_train, mlpc_preds_test, y_train, y_test
)

# Вывод метрик
print(*train_print_mlpc, '-'*25, sep='\n')
print(*test_print_mlpc, '-'*25, sep='\n')
print('Full test metrics report:')
print(metrics.classification_report(y_test, mlpc_preds_test, digits=4))

Train aggregate metrics:
Precision: 1.0
Recall:    1.0
F1 score:  1.0
Accuracy:  1.0
-------------------------
Test aggregate metrics:
Precision: 0.9716
Recall:    0.9705
F1 score:  0.9703
Accuracy:  0.9705
-------------------------
Full test metrics report:
              precision    recall  f1-score   support

         1.0     0.9859    0.9859    0.9859       496
         2.0     0.9769    0.9873    0.9820       471
         3.0     0.9904    0.9810    0.9856       420
         4.0     0.9797    0.8839    0.9293       491
         5.0     0.9064    0.9831    0.9432       532
         6.0     0.9963    1.0000    0.9981       537

    accuracy                         0.9705      2947
   macro avg     0.9726    0.9702    0.9707      2947
weighted avg     0.9716    0.9705    0.9703      2947



In [97]:
# создаём таблицу сопряжённости
ct = pd.crosstab(y_test, mlpc_preds_test)
# определяем название активностей
ct.index = ['ходьба', 'подъём', 
            'спуск', 'сидение',
            'стояние', 'лежание'
]
ct.columns = list(ct.index)
ct

,ходьба,подъём,спуск,сидение,стояние,лежание
ходьба,489,4,3,0,0,0
подъём,5,465,1,0,0,0
спуск,2,6,412,0,0,0
сидение,0,1,0,434,54,2
стояние,0,0,0,9,523,0
лежание,0,0,0,0,0,537


Алгоритм однозначно переобучен, но на данный момент демонстрирует наилучший результат.

### LinearSVC

In [56]:
# Инициализация объекта
lin_svc = svm.LinearSVC(random_state=42)
# Обучение
lin_svc.fit(X_train_pca, y_train)
# Предсказания
lin_svc_preds_train = lin_svc.predict(X_train_pca)
lin_svc_preds_test = lin_svc.predict(X_test_pca)

# Получение метрик
train_print_lin_svc, test_print_lin_svc, train_array_lin_svc, test_array_lin_svc = metrics_extraction(
    lin_svc_preds_train, lin_svc_preds_test, y_train, y_test
)

# Вывод метрик
print(*train_print_lin_svc, '-'*25, sep='\n')
print(*test_print_lin_svc, '-'*25, sep='\n')
print('Full test metrics report:')
print(metrics.classification_report(y_test, lin_svc_preds_test, digits=4))

Train aggregate metrics:
Precision: 0.9977
Recall:    0.9977
F1 score:  0.9977
Accuracy:  0.9977
-------------------------
Test aggregate metrics:
Precision: 0.9614
Recall:    0.9606
F1 score:  0.9605
Accuracy:  0.9606
-------------------------
Full test metrics report:
              precision    recall  f1-score   support

         1.0     0.9916    0.9476    0.9691       496
         2.0     0.9528    0.9851    0.9687       471
         3.0     0.9833    0.9833    0.9833       420
         4.0     0.9452    0.8778    0.9102       491
         5.0     0.9069    0.9699    0.9373       532
         6.0     0.9926    1.0000    0.9963       537

    accuracy                         0.9606      2947
   macro avg     0.9621    0.9606    0.9608      2947
weighted avg     0.9614    0.9606    0.9605      2947



### SVC

In [57]:
# Инициализация объекта
svc = svm.SVC(
    kernel='linear',
    random_state=42
)
# Обучение
svc.fit(X_train_pca, y_train)
# Предсказания
svc_preds_train = svc.predict(X_train_pca)
svc_preds_test = svc.predict(X_test_pca)

# Получение метрик
train_print_svc, test_print_svc, train_array_svc, test_array_svc = metrics_extraction(
    svc_preds_train, svc_preds_test, y_train, y_test
)

# Вывод метрик
print(*train_print_svc, '-'*25, sep='\n')
print(*test_print_svc, '-'*25, sep='\n')
print('Full test metrics report:')
print(metrics.classification_report(y_test, svc_preds_test, digits=4))

Train aggregate metrics:
Precision: 0.9966
Recall:    0.9966
F1 score:  0.9966
Accuracy:  0.9966
-------------------------
Test aggregate metrics:
Precision: 0.9622
Recall:    0.962
F1 score:  0.9619
Accuracy:  0.962
-------------------------
Full test metrics report:
              precision    recall  f1-score   support

         1.0     0.9778    0.9758    0.9768       496
         2.0     0.9583    0.9766    0.9674       471
         3.0     0.9807    0.9667    0.9736       420
         4.0     0.9444    0.9002    0.9218       491
         5.0     0.9150    0.9511    0.9327       532
         6.0     1.0000    1.0000    1.0000       537

    accuracy                         0.9620      2947
   macro avg     0.9627    0.9617    0.9621      2947
weighted avg     0.9622    0.9620    0.9619      2947



Попробуем также оптимизировать алгоритм.

In [58]:
%%time
# Напишем функцию для оптимизации
def optuna_SVC(trial):
    """Функция, обучающая модель LogisticRegression по переданным гиперпараметрам
    Args:
        trial : класс, от которого вызываются гиперпараметры
    Returns:
        score(float): метрика F1
    """
    # Задаем пространство поиска гиперпараметров
    params = {
        'C': trial.suggest_float('C', 0.01, 20),
        'kernel': trial.suggest_categorical('kernel', ['rbf', 'linear']),
        'gamma': trial.suggest_categorical('gamma', ['scale', 'auto']),
        'coef0': trial.suggest_float('C', 0, 1),
        'decision_function_shape': trial.suggest_categorical('decision_function_shape', ['ovo', 'ovr'])
    }
    
    # Создаем модель
    model = svm.SVC(
        **params,
        random_state=42
    )
    # Рассчитываем метрику на кросс-валидации
    score = model_selection.cross_val_score(
        model, X_train_pca, y_train, cv=10, scoring="f1_weighted", n_jobs=-1
    ).mean()
    return score

# Создаем объект исследования для первого набора гиперпараметров
# Укажем, что нам необходимо максимизировать метрику direction="maximize"
study_SVC = optuna.create_study(study_name="SVC_opt", direction="maximize",)
# Подавляем логирование
optuna.logging.set_verbosity(optuna.logging.WARNING)
# Ищем лучшую комбинацию гиперпараметров n_trials раз
study_SVC.optimize(optuna_SVC, n_trials=20)

# Передаем модели коллекцию оптимальных гиперпараметров
opt_SVC = svm.SVC(
    **study_SVC.best_params,
    random_state=42
)
# Обучение
opt_SVC.fit(X_train_pca, y_train)
# Предсказания
preds_train_opt_SVC = opt_SVC.predict(X_train_pca)
preds_test_opt_SVC = opt_SVC.predict(X_test_pca)

# Получение метрик
train_print_opt_SVC, test_print_opt_SVC, train_array_opt_SVC, test_array_opt_SVC = metrics_extraction(
    preds_train_opt_SVC, preds_test_opt_SVC, y_train, y_test
)

# Вывод метрик
print(f'Лучшие гиперпараметры: {study_SVC.best_params}')
print("f1_score на обучающем наборе: {:.4f}\n".format(study_SVC.best_value))
print(*train_print_opt_SVC, '-'*25, sep='\n')
print(*test_print_opt_SVC, '-'*25, sep='\n')
print('Full test metrics report:')
print(metrics.classification_report(y_test, preds_test_opt_SVC, digits=4))

Лучшие гиперпараметры: {'C': 0.6717891413252985, 'kernel': 'linear', 'gamma': 'auto', 'decision_function_shape': 'ovo'}
f1_score на обучающем наборе: 0.9471

Train aggregate metrics:
Precision: 0.9958
Recall:    0.9958
F1 score:  0.9958
Accuracy:  0.9958
-------------------------
Test aggregate metrics:
Precision: 0.9636
Recall:    0.9634
F1 score:  0.9633
Accuracy:  0.9634
-------------------------
Full test metrics report:
              precision    recall  f1-score   support

         1.0     0.9778    0.9758    0.9768       496
         2.0     0.9583    0.9766    0.9674       471
         3.0     0.9807    0.9667    0.9736       420
         4.0     0.9468    0.9063    0.9261       491
         5.0     0.9201    0.9530    0.9363       532
         6.0     1.0000    1.0000    1.0000       537

    accuracy                         0.9634      2947
   macro avg     0.9640    0.9631    0.9634      2947
weighted avg     0.9636    0.9634    0.9633      2947

CPU times: total: 1.12 s
Wal

Получилось неслабо, идем дальше.

### DecisionTreeClassifier

In [123]:
%%time
# Напишем функцию для перебора гиперпараметров
def optuna_dtc(trial):
    """Функция, обучающая модель DecisionTreeClassifier по переданным гиперпараметрам
    Args:
        trial : класс, от которого вызываются гиперпараметры
    Returns:
        score(float): метрика F1
    """
    # Задаем пространствао поиска гиперпараметров
    params = {
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy', 'log_loss']),
        'splitter': trial.suggest_categorical('splitter', ['best', 'random']),
        'max_depth': trial.suggest_int('max_depth', 2, 50),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 51, 2),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20)
    }

    # Создаем модель
    model=tree.DecisionTreeClassifier(
        **params,
        random_state=42        
    )   
    # Рассчитаем метрику на кросс-валидации
    score = model_selection.cross_val_score(
        model, X_train_pca, y_train, cv=10, scoring="f1_weighted", n_jobs=-1
    ).mean()
    return score


# Создаем объект исследования для первого набора гиперпараметров
# Укажем, что нам необходимо максимизировать метрику direction="maximize"
study_dtc = optuna.create_study(study_name="dtc_opt", direction="maximize")
# Подавляем логирование
optuna.logging.set_verbosity(optuna.logging.WARNING)
# Ищем лучшую комбинацию гиперпараметров n_trials раз
study_dtc.optimize(optuna_dtc, n_trials=20)

# Передаем модели коллекцию оптимальных гиперпараметров
opt_dtc = tree.DecisionTreeClassifier(**study_dtc.best_params, random_state=42)
# Обучение
opt_dtc.fit(X_train_pca, y_train)
# Предсказания
preds_train_opt_dtc = opt_dtc.predict(X_train_pca)
preds_test_opt_dtc = opt_dtc.predict(X_test_pca)

# Получение метрик
train_print_opt_dtc, test_print_opt_dtc, train_array_opt_dtc, test_array_opt_dtc = metrics_extraction(
    preds_train_opt_dtc, preds_test_opt_dtc, y_train, y_test
)

# Вывод метрик
print(f'Лучшие гиперпараметры: {study_dtc.best_params}')
print("f1_score на обучающем наборе: {:.4f}\n".format(study_dtc.best_value))
print(*train_print_opt_dtc, '-'*25, sep='\n')
print(*test_print_opt_dtc, '-'*25, sep='\n')
print('Full test metrics report:')
print(metrics.classification_report(y_test, preds_test_opt_dtc, digits=4))

Лучшие гиперпараметры: {'criterion': 'entropy', 'splitter': 'best', 'max_depth': 7, 'min_samples_split': 22, 'min_samples_leaf': 11}
f1_score на обучающем наборе: 0.7704

Train aggregate metrics:
Precision: 0.8861
Recall:    0.8852
F1 score:  0.8854
Accuracy:  0.8852
-------------------------
Test aggregate metrics:
Precision: 0.8003
Recall:    0.7954
F1 score:  0.7966
Accuracy:  0.7954
-------------------------
Full test metrics report:
              precision    recall  f1-score   support

         1.0     0.7661    0.8649    0.8125       496
         2.0     0.8447    0.7856    0.8141       471
         3.0     0.7692    0.7143    0.7407       420
         4.0     0.6667    0.7251    0.6946       491
         5.0     0.7591    0.7462    0.7526       532
         6.0     0.9801    0.9162    0.9471       537

    accuracy                         0.7954      2947
   macro avg     0.7976    0.7920    0.7936      2947
weighted avg     0.8003    0.7954    0.7966      2947

CPU times: tota

У древесных алгоритмов, похоже, действительно проблемы с нашими данными: отставание классификатора Catboost имеет под собой принципиальное основание.

### RandomForestClassifier

In [41]:
rfc = ensemble.RandomForestClassifier(
    criterion='entropy',
    n_estimators=475,
    max_depth=42,
    min_samples_leaf=2,
    min_samples_split=6,
    random_state=42
)
# Обучение
rfc.fit(X_train_pca, y_train)
# Предсказания
rfc_preds_train = rfc.predict(X_train_pca)
rfc_preds_test = rfc.predict(X_test_pca)

# Получение метрик
train_print_rfc, test_print_rfc, train_array_rfc, test_array_rfc = metrics_extraction(
    rfc_preds_train, rfc_preds_test, y_train, y_test
)

# Вывод метрик
print(*train_print_rfc, '-'*25, sep='\n')
print(*test_print_rfc, '-'*25, sep='\n')
print('Full test metrics report:')
print(metrics.classification_report(y_test, rfc_preds_test, digits=4))

Train aggregate metrics:
Precision: 1.0
Recall:    1.0
F1 score:  1.0
Accuracy:  1.0
-------------------------
Test aggregate metrics:
Precision: 0.906
Recall:    0.9046
F1 score:  0.9041
Accuracy:  0.9046
-------------------------
Full test metrics report:
              precision    recall  f1-score   support

         1.0     0.8872    0.9194    0.9030       496
         2.0     0.9177    0.9236    0.9206       471
         3.0     0.8635    0.8286    0.8457       420
         4.0     0.9233    0.8086    0.8621       491
         5.0     0.8508    0.9436    0.8948       532
         6.0     0.9851    0.9832    0.9842       537

    accuracy                         0.9046      2947
   macro avg     0.9046    0.9011    0.9017      2947
weighted avg     0.9060    0.9046    0.9041      2947



Параметры были подобраны на optuna, весь процесс занял сорок минут времени, а результат все еще хуже, чем у линейных алгоритмов.

### GradientBoostingClassifier

In [42]:
%%time
# Инициализация объекта
gbc = ensemble.GradientBoostingClassifier(
    random_state=42
)
# Обучение
gbc.fit(X_train_pca, y_train)
# Предсказания
gbc_preds_train = gbc.predict(X_train_pca)
gbc_preds_test = gbc.predict(X_test_pca)
# Получение метрик
train_print_gbc, test_print_gbc, train_array_gbc, test_array_gbc = metrics_extraction(
    gbc_preds_train, gbc_preds_test, y_train, y_test
)
# Вывод метрик
print(*train_print_gbc, '-'*25, sep='\n')
print(*test_print_gbc, '-'*25, sep='\n')
print('Full test metrics report:')
print(metrics.classification_report(y_test, gbc_preds_test, digits=4))

Train aggregate metrics:
Precision: 0.9984
Recall:    0.9984
F1 score:  0.9984
Accuracy:  0.9984
-------------------------
Test aggregate metrics:
Precision: 0.9232
Recall:    0.9233
F1 score:  0.9229
Accuracy:  0.9233
-------------------------
Full test metrics report:
              precision    recall  f1-score   support

         1.0     0.9111    0.9294    0.9202       496
         2.0     0.9095    0.9597    0.9339       471
         3.0     0.8966    0.8262    0.8600       420
         4.0     0.9018    0.8982    0.9000       491
         5.0     0.9134    0.9117    0.9125       532
         6.0     0.9963    0.9963    0.9963       537

    accuracy                         0.9233      2947
   macro avg     0.9214    0.9202    0.9205      2947
weighted avg     0.9232    0.9233    0.9229      2947

CPU times: total: 21min 42s
Wall time: 21min 52s


Пожалуй, оптимизация градиентного бустинга займет довольно много времени тоже, рассмотрим лучше его модификацию.

### HistGradientBoostingClassifier

In [43]:
hgbc = ensemble.HistGradientBoostingClassifier(
    random_state=42
)
# Обучение
hgbc.fit(X_train_pca, y_train)
# Предсказания
hgbc_preds_train = hgbc.predict(X_train_pca)
hgbc_preds_test = hgbc.predict(X_test_pca)
# Получение метрик
train_print_hgbc, test_print_hgbc, train_array_hgbc, test_array_hgbc = metrics_extraction(
    hgbc_preds_train, hgbc_preds_test, y_train, y_test
)
# Вывод метрик
print(*train_print_hgbc, '-'*25, sep='\n')
print(*test_print_hgbc, '-'*25, sep='\n')
print('Full test metrics report:')
print(metrics.classification_report(y_test, hgbc_preds_test, digits=4))

Train aggregate metrics:
Precision: 1.0
Recall:    1.0
F1 score:  1.0
Accuracy:  1.0
-------------------------
Test aggregate metrics:
Precision: 0.9299
Recall:    0.9294
F1 score:  0.929
Accuracy:  0.9294
-------------------------
Full test metrics report:
              precision    recall  f1-score   support

         1.0     0.9178    0.9456    0.9315       496
         2.0     0.9194    0.9682    0.9431       471
         3.0     0.9217    0.8405    0.8792       420
         4.0     0.9208    0.8758    0.8977       491
         5.0     0.8984    0.9474    0.9222       532
         6.0     0.9962    0.9814    0.9887       537

    accuracy                         0.9294      2947
   macro avg     0.9290    0.9265    0.9271      2947
weighted avg     0.9299    0.9294    0.9290      2947



In [44]:
%%time
# Напишем функцию для перебора гиперпараметров
def optuna_hgbc(trial):
    """Функция, обучающая модель HistGradientBoostingClassifier по переданным гиперпараметрам
    Args:
        trial : класс, от которого вызываются гиперпараметры
    Returns:
        score(float): метрика F1
    """
    # Задаем пространствао поиска гиперпараметров
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-2, 1),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 2, 50, 2),
        'max_depth': trial.suggest_int('max_depth', 1, 30, 1),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 3, 101, 2),
        'l2_regularization': trial.suggest_float('l2_regularization', 0, 1)
    }

    
    # Создаем модель
    model=ensemble.HistGradientBoostingClassifier(
        **params,
        max_iter=300,
        random_state=42
    )
    # Рассчитаем метрику на кросс-валидации
    score = model_selection.cross_val_score(
        model, X_train_pca, y_train, cv=10, scoring="f1_weighted", n_jobs=-1
    ).mean()
    return score


# Создаем объект исследования для первого набора гиперпараметров
# Укажем, что нам необходимо максимизировать метрику direction="maximize"
study_hgbc = optuna.create_study(study_name="hgbc_opt", direction="maximize")
# Подавляем логирование
optuna.logging.set_verbosity(optuna.logging.WARNING)
# Ищем лучшую комбинацию гиперпараметров n_trials раз
study_hgbc.optimize(optuna_hgbc, n_trials=50)

# Передаем модели коллекцию оптимальных гиперпараметров
opt_hgbc = ensemble.HistGradientBoostingClassifier(
    **study_hgbc.best_params,
    max_iter=300,
    random_state=42
)
# Обучение
opt_hgbc.fit(X_train_pca, y_train)
# Предсказания
preds_train_opt_hgbc = opt_hgbc.predict(X_train_pca)
preds_test_opt_hgbc = opt_hgbc.predict(X_test_pca)

# Получение метрик
train_print_opt_hgbc, test_print_opt_hgbc, train_array_opt_hgbc, test_array_opt_hgbc = metrics_extraction(
    preds_train_opt_hgbc, preds_test_opt_hgbc, y_train, y_test
)

# Вывод метрик
print(f'Лучшие гиперпараметры: {study_hgbc.best_params}')
print("f1_score на обучающем наборе: {:.4f}\n".format(study_hgbc.best_value))
print(*train_print_opt_hgbc, '-'*25, sep='\n')
print(*test_print_opt_hgbc, '-'*25, sep='\n')
print('Full test metrics report:')
print(metrics.classification_report(y_test, preds_test_opt_hgbc, digits=4))

Лучшие гиперпараметры: {'learning_rate': 0.5946483853506327, 'max_leaf_nodes': 2, 'max_depth': 30, 'min_samples_leaf': 29, 'l2_regularization': 0.6127862016309231}
f1_score на обучающем наборе: 0.9231

Train aggregate metrics:
Precision: 1.0
Recall:    1.0
F1 score:  1.0
Accuracy:  1.0
-------------------------
Test aggregate metrics:
Precision: 0.9525
Recall:    0.9525
F1 score:  0.9523
Accuracy:  0.9525
-------------------------
Full test metrics report:
              precision    recall  f1-score   support

         1.0     0.9660    0.9738    0.9699       496
         2.0     0.9599    0.9660    0.9630       471
         3.0     0.9500    0.9500    0.9500       420
         4.0     0.9312    0.8819    0.9059       491
         5.0     0.9144    0.9436    0.9288       532
         6.0     0.9926    0.9963    0.9944       537

    accuracy                         0.9525      2947
   macro avg     0.9523    0.9519    0.9520      2947
weighted avg     0.9525    0.9525    0.9523      29

Оптимизация шла более часа, но оно того стоило.

### XGBClassifier

In [47]:
le = preprocessing.LabelEncoder()
y_train_le = le.fit_transform(y_train)
y_test_le = le.fit_transform(y_test)

xgbc = xgb.XGBClassifier()
# Обучение
xgbc.fit(X_train_pca, y_train_le)
# Предсказания
xgbc_preds_train = xgbc.predict(X_train_pca)
xgbc_preds_test = xgbc.predict(X_test_pca)
# Получение метрик
train_print_xgbc, test_print_xgbc, train_array_xgbc, test_array_xgbc = metrics_extraction(
    xgbc_preds_train, xgbc_preds_test, y_train_le, y_test_le
)
# Вывод метрик
print(*train_print_xgbc, '-'*25, sep='\n')
print(*test_print_xgbc, '-'*25, sep='\n')
print('Full test metrics report:')
print(metrics.classification_report(y_test_le, xgbc_preds_test, digits=4))

Train aggregate metrics:
Precision: 1.0
Recall:    1.0
F1 score:  1.0
Accuracy:  1.0
-------------------------
Test aggregate metrics:
Precision: 0.9266
Recall:    0.9264
F1 score:  0.9259
Accuracy:  0.9264
-------------------------
Full test metrics report:
              precision    recall  f1-score   support

           0     0.9150    0.9335    0.9242       496
           1     0.9157    0.9682    0.9412       471
           2     0.9115    0.8333    0.8706       420
           3     0.9188    0.8758    0.8968       491
           4     0.8961    0.9398    0.9174       532
           5     0.9962    0.9888    0.9925       537

    accuracy                         0.9264      2947
   macro avg     0.9255    0.9232    0.9238      2947
weighted avg     0.9266    0.9264    0.9259      2947



In [50]:
%%time
# Напишем функцию для перебора гиперпараметров
def optuna_xgbc(trial):
    """Функция, обучающая модель XGBClassifier по переданным гиперпараметрам
    Args:
        trial : класс, от которого вызываются гиперпараметры
    Returns:
        score(float): метрика F1
    """
    # Задаем пространствао поиска гиперпараметров
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 18, 1),
        'gamma': trial.suggest_float('gamma', 1, 9),
        'reg_alpha': trial.suggest_int('reg_alpha', 40, 180, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1),
        'min_child_weight': trial.suggest_int('min_child_weight', 0, 10, 1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1001, 25)
    }

    # Создаем модель
    model=xgb.XGBClassifier(
        **params,
        seed=42
    )
    # Рассчитаем метрику на кросс-валидации
    score = model_selection.cross_val_score(
        model, X_train_pca, y_train_le, cv=10, scoring="f1_weighted", n_jobs=-1
    ).mean()
    return score

# Создаем объект исследования для первого набора гиперпараметров
# Укажем, что нам необходимо максимизировать метрику direction="maximize"
study_xgbc = optuna.create_study(study_name="xgbc_opt", direction="maximize")
# Подавляем логирование
optuna.logging.set_verbosity(optuna.logging.WARNING)
# Ищем лучшую комбинацию гиперпараметров n_trials раз
study_xgbc.optimize(optuna_xgbc, n_trials=30)

# Передаем модели коллекцию оптимальных гиперпараметров
opt_xgbc = xgb.XGBClassifier(**study_xgbc.best_params)
# Обучение
opt_xgbc.fit(X_train_pca, y_train_le)
# Предсказания
preds_train_opt_xgbc = opt_xgbc.predict(X_train_pca)
preds_test_opt_xgbc = opt_xgbc.predict(X_test_pca)

# Получение метрик
train_print_opt_xgbc, test_print_opt_xgbc, train_array_opt_xgbc, test_array_opt_xgbc = metrics_extraction(
    preds_train_opt_xgbc, preds_test_opt_xgbc, y_train_le, y_test_le
)

# Вывод метрик
print(f'Лучшие гиперпараметры: {study_xgbc.best_params}')
print("f1_score на обучающем наборе: {:.4f}\n".format(study_xgbc.best_value))
print(*train_print_opt_xgbc, '-'*25, sep='\n')
print(*test_print_opt_xgbc, '-'*25, sep='\n')
print('Full test metrics report:')
print(metrics.classification_report(y_test_le, preds_test_opt_xgbc, digits=4))

Лучшие гиперпараметры: {'max_depth': 15, 'gamma': 1.056683066713876, 'reg_alpha': 41, 'reg_lambda': 0.838433409752353, 'colsample_bytree': 0.9402579732543315, 'min_child_weight': 8, 'n_estimators': 300}
f1_score на обучающем наборе: 0.8420

Train aggregate metrics:
Precision: 0.9538
Recall:    0.9538
F1 score:  0.9536
Accuracy:  0.9538
-------------------------
Test aggregate metrics:
Precision: 0.8845
Recall:    0.8843
F1 score:  0.8831
Accuracy:  0.8843
-------------------------
Full test metrics report:
              precision    recall  f1-score   support

           0     0.8505    0.9173    0.8826       496
           1     0.8735    0.9236    0.8978       471
           2     0.8520    0.7262    0.7841       420
           3     0.8723    0.8208    0.8458       491
           4     0.8523    0.9004    0.8757       532
           5     0.9944    0.9851    0.9897       537

    accuracy                         0.8843      2947
   macro avg     0.8825    0.8789    0.8793      2947


Похоже, оптимизатор и на этот раз застрял в локальном минимуме.

### StackingClassifier

In [83]:
# Список моделей для стэка
estimators = [
    ('log_reg', LogR_opt),
    ('pac', pac),
    ('ridge', ridge),
    ('mlpc', mlpc),
    ('lin_svc', lin_svc),
    ('opt_SVC', opt_SVC)
]
# Инициализация объекта
sc_metamodel = ensemble.StackingClassifier(
    estimators=estimators,
    final_estimator=linear_model.LogisticRegression(
        random_state=42
    ),
    n_jobs=-1
)
# Обучение
sc_metamodel.fit(X_train_pca, y_train)
# Предсказания
preds_train_scmm = sc_metamodel.predict(X_train_pca)
preds_test_scmm = sc_metamodel.predict(X_test_pca)

# Получение метрик
train_print_scmm, test_print_scmm, train_array_scmm, test_array_scmm = metrics_extraction(
    preds_train_scmm, preds_test_scmm, y_train, y_test
)

# Вывод метрик
print(*train_print_scmm, '-'*25, sep='\n')
print(*test_print_scmm, '-'*25, sep='\n')
print('Full test metrics report:')
print(metrics.classification_report(y_test, preds_test_scmm, digits=4))

Train aggregate metrics:
Precision: 0.9934
Recall:    0.9933
F1 score:  0.9933
Accuracy:  0.9933
-------------------------
Test aggregate metrics:
Precision: 0.9722
Recall:    0.9715
F1 score:  0.9714
Accuracy:  0.9715
-------------------------
Full test metrics report:
              precision    recall  f1-score   support

         1.0     0.9818    0.9778    0.9798       496
         2.0     0.9788    0.9788    0.9788       471
         3.0     0.9905    0.9976    0.9941       420
         4.0     0.9714    0.9002    0.9345       491
         5.0     0.9153    0.9756    0.9445       532
         6.0     1.0000    1.0000    1.0000       537

    accuracy                         0.9715      2947
   macro avg     0.9730    0.9717    0.9719      2947
weighted avg     0.9722    0.9715    0.9714      2947



## Итог

Проведем стекинг-алгоритм через кросс-валидацию.

In [97]:
cv_scores(sc_metamodel, X, y)

Train scores:
Precision:   0.9983
Recall:      0.9983
F1 weighted: 0.9983
Accuracy:    0.9983
Test scores:
Precision:   0.9671
Recall:      0.9638
F1 weighted: 0.9635
Accuracy:    0.9638


Также проведем через кросс-валидацию каждую из моделей из состава стекинга.

In [96]:
models = [LogR_opt, pac, ridge, mlpc, lin_svc, opt_SVC]

for model in models:
    print(f'Model: {model}')
    cv_scores(model, X, y)
    print('-'*20)

Model: LogisticRegression(C=0.7630868925337111, max_iter=3000, n_jobs=-1,
                   random_state=42, solver='newton-cholesky')
Train scores:
Precision:   0.9934
Recall:      0.9934
F1 weighted: 0.9934
Accuracy:    0.9934
Test scores:
Precision:   0.9652
Recall:      0.9615
F1 weighted: 0.9612
Accuracy:    0.9615
--------------------
Model: PassiveAggressiveClassifier(random_state=42)
Train scores:
Precision:   0.9887
Recall:      0.9885
F1 weighted: 0.9885
Accuracy:    0.9885
Test scores:
Precision:   0.9608
Recall:      0.9569
F1 weighted: 0.9568
Accuracy:    0.9569
--------------------
Model: RidgeClassifier(random_state=42, solver='lsqr')
Train scores:
Precision:   0.9813
Recall:      0.9813
F1 weighted: 0.9813
Accuracy:    0.9813
Test scores:
Precision:   0.9612
Recall:      0.9582
F1 weighted: 0.958
Accuracy:    0.9582
--------------------
Model: MLPClassifier(activation='tanh', hidden_layer_sizes=(166, 36), random_state=42)
Train scores:
Precision:   1.0
Recall:      1.0

Наилучший результат продемонстрировал MLPClassifier.